test case 1: Talk with model

In [4]:
from llama_index.llms.ollama import Ollama
llm = Ollama(model="gemma:2b")

resp = llm.complete("What did Rome grow? Be concise.")
print(resp)

Rome grew into a powerful city-state in the Italian Peninsula, with a rich history and culture.


test case 2: Simple conversation

In [1]:
from llama_index.llms.ollama import Ollama
from llama_index.core import ServiceContext
from llama_index.core.chat_engine import SimpleChatEngine

llm = Ollama(model="llama3.1")

service_context = ServiceContext.from_defaults(
    llm=llm,
    embed_model="local:BAAI/bge-small-en-v1.5", 
)

chat_engine = SimpleChatEngine.from_defaults(service_context=service_context)


/tmp/ipykernel_20121/4177303745.py:7: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(
/home/rajatdulal/miniconda3/envs/langchain/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# print(chat_engine.chat("Hi, my name is Mirna"))


In [3]:
# print(chat_engine.chat("Can you tell what my name is?"))

In [4]:
print(chat_engine.chat("""The Indo-Pacific is becoming a key battleground for influence, especially with the rise of China. 
                       Their assertive stance in the South China Sea, coupled with their Belt and Road Initiative, is reshaping regional power structures. 
                       Nations like India, Japan, and Australia are responding by strengthening their own ties, seeking to counterbalance China’s influence."""))

**Analysis:**

The passage highlights the growing influence of China in the Indo-Pacific region, particularly in the South China Sea. The rise of China's assertiveness and its Belt and Road Initiative is impacting regional power structures and leading other nations to strengthen their own ties and seek counterbalancing influence.

**Key Points:**

* **China's Assertiveness:** China's assertive stance in the South China Sea and its Belt and Road Initiative are reshaping the region's power structures.
* **India, Japan, and Australia's Response:** India, Japan, and Australia are responding by strengthening their own ties and seeking to counterbalance China's influence.
* **Counterbalancing China's Influence:** Other nations are working together to establish a more balanced and peaceful order in the Indo-Pacific.

**Additional Insights:**

* The passage also suggests that the Indo-Pacific region is becoming a major battleground for global power competition.
* The rise of China is transform

In [5]:
print(chat_engine.chat("Which countries are looking to counterbalance China?"))

The passage mentions the following countries that are looking to counterbalance China's influence:

* India
* Japan
* Australia


#test 3: Simple RAG

In [4]:
from llama_index.core import (
    ServiceContext,
    SimpleDirectoryReader,
    StorageContext,
    VectorStoreIndex,
    set_global_service_context,
)
from llama_index.llms.ollama import Ollama

llm = Ollama(model="gemma:2b")


In [5]:

# Reads pdfs at "./" path
documents = (
    SimpleDirectoryReader(
        input_dir = './',
        required_exts = [".pdf"])
        .load_data()
)

# ServiceContext is a bundle of commonly used 
# resources used during the indexing and 
# querying stage 
service_context = (
    ServiceContext
    .from_defaults(
        llm=llm, 
        embed_model="local:BAAI/bge-small-en-v1.5", 
        chunk_size=300
    )
)
set_global_service_context(service_context)

# Node represents a “chunk” of a source Document
nodes = (
    service_context
    .node_parser
    .get_nodes_from_documents(documents)
)

# offers core abstractions around storage of Nodes, 
# indices, and vectors
storage_context = StorageContext.from_defaults()
storage_context.docstore.add_documents(nodes)

# Create the vectorstore index
index = (
    VectorStoreIndex
    .from_documents(
        documents, 
        storage_context=storage_context, 
        llm=llm
        )
)
query_engine = index.as_query_engine()


/tmp/ipykernel_20187/4212152171.py:14: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  .from_defaults(


In [6]:

# Query the index
query="""Which countries are trying to counterbalance the China?"""
response = query_engine.query(query)
print(response)

Sure. The countries that are trying to counterbalance China are India, Japan, and Australia.


In [8]:

# Query the index
query="""How are some countries trying to counterbalance the China?"""
response = query_engine.query(query)
print(response)

Sure, here's the answer to the query:

Some countries are strengthening their own ties and seeking to counterbalance China's influence. This is evident in the Quad, which is a grouping of the United States, India, Japan, and Australia, who are working to maintain peace and stability in the Indo-Pacific region. The Quad aims to prevent a military confrontation between China and its allies. Additionally, countries like India and Japan are strengthening their partnerships with each other to counter China's influence.


In [ ]:
# from llama_index.llms.ollama import Ollama
# from llama_index.embeddings.huggingface import HuggingFaceEmbedding
# from llama_index.core import Settings

# Settings.llm = Ollama(model="llama2", request_timeout=120.0)
# Settings.embed_model = HuggingFaceEmbedding(
#     model_name="BAAI/bge-small-en-v1.5"
# )

#task 4:Simple agent

In [12]:

from llama_index.llms.ollama import Ollama
from llama_index.core.agent import ReActAgent
from llama_index.core.tools import FunctionTool

def add_numbers_three_fn(a : int, b: int) -> int:
    """Adds two numbers and a constant 3 and returns the result"""
    return a+b+3

tools = [
    FunctionTool.from_defaults(fn=add_numbers_three_fn)
]

llm = Ollama(model="gemma:2b")
agent = ReActAgent.from_tools(tools, llm=llm, verbose=True)
response = agent.chat("Add the numbers 3 and 2")
print(str(response))

> Running step 76d68232-eeab-417c-81a7-928f01942214. Step input: Add the numbers 3 and 2
Thought: (Implicit) I can answer without any more tools!
Answer: **Observation:** tool response
```
Observation: add_numbers_three_fn
```
**Observation:** tool response
```
Observation: add_numbers_three_fn
```


Example #5 — Conversational RAG
Here, you will do the same from Example #3, using a different method when instantiating the chat_engine object, that isindex.as_chat_engine instead of index.as_query_engine.

Now, you can ask follow-up questions, and the chat engine will be able to handle memory for you.

In [1]:
from llama_index.core import (
    ServiceContext,
    SimpleDirectoryReader,
    StorageContext,
    VectorStoreIndex,
    set_global_service_context,
)
from llama_index.llms.ollama import Ollama

llm = Ollama(model="gemma:2b")

# Reads pdfs at "./" path
documents = (
    SimpleDirectoryReader(
        input_dir = './',
        required_exts = [".pdf"])
        .load_data()
)

# ServiceContext is a bundle of commonly used 
# resources used during the indexing and 
# querying stage 
service_context = (
    ServiceContext
    .from_defaults(
        llm=llm, 
        embed_model="local:BAAI/bge-small-en-v1.5", 
        chunk_size=300
    )
)
set_global_service_context(service_context)

# Node represents a “chunk” of a source Document
nodes = (
    service_context
    .node_parser
    .get_nodes_from_documents(documents)
)

# offers core abstractions around storage of Nodes, 
# indices, and vectors
storage_context = StorageContext.from_defaults()
storage_context.docstore.add_documents(nodes)

# Create the vectorstore index
index = (
    VectorStoreIndex
    .from_documents(
        documents, 
        storage_context=storage_context, 
        llm=llm
        )
)


/tmp/ipykernel_22447/3573117386.py:25: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  .from_defaults(
/home/rajatdulal/miniconda3/envs/langchain/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:

chat_engine = index.as_chat_engine(chat_mode="context", verbose = True)


In [ ]:
response = chat_engine.chat("Hi, my name is Rajat")
print(response)

In [15]:

#Assistant: Hello Mirna! It's great to meet you. What can I help with today?
response = chat_engine.chat("What is my name?")
print(response)


Your name is Rajat. It's nice to meet you too!


In [4]:

# Query the index
query="""please give insights on the conversation between Ram and Sita"""
response = chat_engine.chat(query)
print(response)


I'm unable to access external information or engage in real-time conversations, so I cannot provide insights on the conversation between Ram and Sita.
